In [2]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("FlightPipeline").getOrCreate()

# Check Spark session details
print("Spark Session Initialized:")
print(f"App Name: {spark.sparkContext.appName}")
print(f"Master: {spark.sparkContext.master}")
print(f"Version: {spark.version}")


Spark Session Initialized:
App Name: FlightPipeline
Master: local[*]
Version: 3.5.3


In [8]:
from FlightRadar24 import FlightRadar24API
fr_api = FlightRadar24API()

In [9]:
flights = fr_api.get_flights() 

In [4]:
flights = fr_api.get_flights()  # Returns a list of Flight objects
airports = fr_api.get_airports()  # Returns a list of Airport objects
airlines = fr_api.get_airlines()
zones = fr_api.get_zones()


### FLIGHTS

In [102]:
for i in flights[25:30] : 
    flight_details = fr_api.get_flight_details(i)
    flight.set_flight_details(flight_details)
    flight.destination_airport_name
    print("Flying to", flight.destination_airport_name)

Flying to N/A
Flying to Atlanta Hartsfield-Jackson International Airport
Flying to Anchorage Ted Stevens International Airport
Flying to New York John F. Kennedy International Airport
Flying to London Heathrow Airport


In [3]:
print(dir(flights[0])) 

['_Flight__get_info', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_default_text', 'aircraft_code', 'aircraft_text', 'airline_iata', 'airline_icao', 'altitude', 'callsign', 'check_info', 'destination_airport_iata', 'get_altitude', 'get_distance_from', 'get_flight_level', 'get_ground_speed', 'get_heading', 'get_vertical_speed', 'ground_speed', 'heading', 'icao_24bit', 'id', 'latitude', 'longitude', 'number', 'on_ground', 'origin_airport_iata', 'registration', 'set_flight_details', 'squawk', 'time', 'vertical_speed']


In [10]:
import pandas as pd

# Retrieve all flights


# Initialize a list to hold the extracted data
data = []

# Loop over all flights
for flight in flights:
        # Retrieve flight details for each flight
    try:
        # Extract relevant fields
        data.append({
            "flightID": flight.id,
            "aircraft": flight.aircraft_code,
            "airlineCode": flight.airline_icao,
            "airportOrigineCode": flight.origin_airport_iata,
            "airportDestinationCode": flight.destination_airport_iata,
            "flightStatus": flight.on_ground
        })
    except Exception as e:
        # Handle errors gracefully
        print(f"Error retrieving flight details for flight {flight}: {e}")

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df


,flightID,aircraft,airlineCode,airportOrigineCode,airportDestinationCode,flightStatus
0,37ef8f6b,GLID,,,,0
1,37f1ce38,GLID,,,,0
2,38004d6c,GLID,,,,0
3,3802d179,GRND,,,,1
4,3802d2f7,GRND,,,,1
...,...,...,...,...,...,...
1495,3856eb8c,A320,PIA,AUH,PEW,0
1496,3856ebb3,DH8C,ANG,POM,POM,1
1497,3856ebdb,A320,JBU,JFK,SJO,0
1498,3856ec17,B789,QFA,HNL,SYD,0


In [11]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Define schema
schema = StructType([
    StructField("flightID", StringType(), True),
    StructField("aircraft", StringType(), True),
    StructField("airlineCode", StringType(), True),
    StructField("airportOrigineCode", StringType(), True),
    StructField("airportDestinationCode", StringType(), True),
    StructField("flightStatus", IntegerType(), True)
])

# Clean the data
cleaned_data = [
    {k: (v if v is not None else "") for k, v in record.items()}
    for record in data
]

# Create DataFrame with the cleaned data
df_flights = spark.createDataFrame(cleaned_data, schema=schema)

# Show the DataFrame
df_flights.show()


AttributeError: 'NoneType' object has no attribute 'sc'

In [9]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("flightID", StringType(), True),
    StructField("aircraft", StringType(), True),
    StructField("airlineCode", StringType(), True),
    StructField("airportOrigineCode", StringType(), True),
    StructField("airportDestinationCode", StringType(), True),
    StructField("flightStatus", IntegerType(), True)
])

df_flights = spark.createDataFrame(data, schema=schema)
df_flights.show()



Py4JJavaError: An error occurred while calling o58.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (host.docker.internal executor driver): java.net.SocketException: Connection reset
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:186)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:140)
	at java.base/java.io.BufferedInputStream.fill(BufferedInputStream.java:252)
	at java.base/java.io.BufferedInputStream.read(BufferedInputStream.java:271)
	at java.base/java.io.DataInputStream.readInt(DataInputStream.java:392)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:774)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext

In [19]:
# filter out the airplanes that are on the ground and we filter out GLID flights because they doint give any sence, plus take of the drones because its not an airplane
missing_values_df = df[df.isnull().any(axis=1) | (df == "").any(axis=1)]

missing_values_df.query("flightStatus==0 and aircraft !='GLID' and aircraft!='DRON'")

,flightID,aircraft,airlineCode,airportOrigineCode,airportDestinationCode,flightStatus
24,38527368,B738,PBD,,,0
26,38535b87,A321,LMU,SSH,,0
29,3853f5b9,B738,PBD,IKT,,0
31,38541a48,B744,,HKG,,0
35,38542b51,A332,,,,0
...,...,...,...,...,...,...
1451,3855615a,GLEX,,,,0
1453,385561a4,C172,BZQ,YPQ,,0
1458,3855625a,,,POB,,0
1474,385563bc,A321,LMU,,,0


In [24]:
df = df[(df != "").all(axis=1)]


In [25]:
df

,flightID,aircraft,airlineCode,airportOrigineCode,airportDestinationCode,flightStatus
25,38533d7d,A359,AAR,ICN,LHR,0
27,38539d80,A359,AAR,LAX,ICN,0
28,3853f44d,A388,KAL,JFK,ICN,0
30,38541784,B789,ANZ,JFK,AKL,0
32,38541f21,A35K,QTR,AKL,DOH,0
...,...,...,...,...,...,...
1495,385566be,A35K,CPA,HKG,CHC,0
1496,385566cd,B77W,UAE,DUB,DXB,0
1497,385566df,B788,ETH,LFW,ADD,1
1498,385566e5,B77W,AFR,CDG,LAX,0


In [42]:
# Extract the first letter from 'aircraft' column, ignoring 'NA' and empty strings
df['FirstLetter'] = df['aircraft'].apply(lambda x: x[0] if pd.notna(x) and len(x) > 0 else None)

# Drop rows where 'FirstLetter' is None
df = df[df['FirstLetter'].notna()]

# Display the DataFrame
df


C:\Users\tefte\AppData\Local\Temp\ipykernel_20660\1461359894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FirstLetter'] = df['aircraft'].apply(lambda x: x[0] if pd.notna(x) and len(x) > 0 else None)


,flightID,aircraft,airlineCode,airportOrigineCode,airportDestinationCode,flightStatus,FirstLetter
0,37ef8f6b,GLID,,,,0,G
1,37f1ce38,GLID,,,,0,G
2,38004d6c,GLID,,,,0,G
3,3802d179,GRND,,,,1,G
4,3802d2f7,GRND,,,,1,G
...,...,...,...,...,...,...,...
1495,38547ec4,A388,UAE,DXB,SFO,0,A
1496,38547ed6,B77L,CSN,PVG,SEA,0,B
1497,38547edb,B77W,UAE,DAC,DXB,0,B
1498,38547ee6,B752,,UIO,ATL,0,B


In [43]:
# Get distinct values of 'FirstLetter'
distinct_first_letters = df['FirstLetter'].unique()

# Display the distinct values
print(distinct_first_letters)


['G' 'T' 'S' 'B' 'E' 'A' 'C' 'D' 'M' 'F' 'I' 'P' 'K' 'H']


In [48]:
# Filter rows where 'FirstLetter' is 'C'
filtered_df = df[df['FirstLetter'] == 'S']

# Display the filtered DataFrame
print(filtered_df)


      flightID aircraft airlineCode airportOrigineCode airportDestinationCode  \
23    384d9c36     SHIP                                                         
42    3852e3e0     SU95         AZO                                             
326   3853f38c     SR20                            LYA                          
944   385455ff     SU95         GZP                                             
1119  3854673b     SU95         GZP                                             
1153  385469b4     SU95         GZP                                             
1163  38546ab0     SU95         RWZ                MCX                    KUF   

      flightStatus FirstLetter  
23               0           S  
42               0           S  
326              0           S  
944              0           S  
1119             0           S  
1153             0           S  
1163             0           S  


In [ ]:
import pandas as pd

# Retrieve all flights


# Initialize a list to hold the extracted data
data = []

# Loop over all flights
for flight in flights:
        # Retrieve flight details for each flight
    try:
        flight_details = fr_api.get_flight_details(flight)  
        # Extract relevant fields
        data.append({
            "aircraft": flight_details['aircraft']['model']['text'],
            # "airportOrigineLat": flight_details['airport']['origin']['position']['latitude'],
            # "airportOrigineLong": flight_details['airport']['origin']['position']['longitude'],
            # "airportOrigineCountryID": flight_details['airport']['origin']['position']['country']['id'],
            # "airportOrigineContinent": flight_details['airport']['origin']['timezone']['name'],
            # "airportDestinationLat": flight_details['airport']['destination']['position']['latitude'],
            # "airportDestinationLong": flight_details['airport']['destination']['position']['longitude'],
            # "airportDestinationCountryID": flight_details['airport']['destination']['position']['country']['id'],
            # "airportDestinationContinent": flight_details['airport']['destination']['timezone']['name']
        })
    except Exception as e:
        # Handle errors gracefully
        print(f"Error retrieving flight details for flight {flight}: {e}")

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
df


In [141]:
# Count occurrences of each airline in the 'Airline' column
airline_counts = df['Airline'].value_counts()

# Convert to a DataFrame for better visualization
df_airline_counts = airline_counts.reset_index()
df_airline_counts.columns = ['Airline', 'Count']

# Display the DataFrame
print(df_airline_counts)


                             Airline  Count
0                    United Airlines     28
1                           Emirates     18
2                      Qatar Airways     18
3                    Delta Air Lines     16
4                   Turkish Airlines     15
..                               ...    ...
180                  Virgin Atlantic      1
181                          Avianca      1
182                Discover Airlines      1
183  Jeju Air (Zanmang Loopy Livery)      1
184         Condor (Blue Sea Livery)      1

[185 rows x 2 columns]


### airports

In [3]:
airports = fr_api.get_airports() 

In [8]:
print(dir(airports[0])) 

['_Airport__get_info', '_Airport__initialize_with_basic_info', '_Airport__initialize_with_info', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_default_text', 'altitude', 'country', 'get_distance_from', 'iata', 'icao', 'latitude', 'longitude', 'name', 'set_airport_details']


In [14]:
airports[1].icao

'EDKA'

In [15]:
airport_details = fr_api.get_airport_details('EDKA')

In [16]:
airport_details

{'airport': {'pluginData': {'details': {'name': 'Aachen Merzbruck Airport',
    'code': {'iata': 'AAH', 'icao': 'EDKA'},
    'delayIndex': {'arrivals': None, 'departures': None},
    'stats': None,
    'position': {'latitude': 50.821899,
     'longitude': 6.184759,
     'elevation': 626,
     'country': {'name': 'Germany', 'code': 'DE', 'id': 83},
     'region': {'city': 'Aachen'}},
    'timezone': {'name': 'Europe/Berlin',
     'offset': 3600,
     'abbr': 'CET',
     'abbrName': 'Central European Time',
     'isDst': False},
    'url': {'homepage': None,
     'webcam': 'http://airportwebcams.net/aachen-merzbruck-airfield-webcam/',
     'wikipedia': 'https://en.wikipedia.org/wiki/Aachen_Merzbr%C3%BCck_Airfield'},
    'airportImages': {'thumbnails': [{'src': 'https://cdn.jetphotos.com/200/6/39851_1516008127_tb.jpg',
       'link': 'https://www.jetphotos.com/photo/8826727',
       'copyright': 'C. v. Grinsven',
       'source': 'Jetphotos.com'}],
     'medium': [{'src': 'https://cdn.jet

In [ ]:
import pandas as pd

# Retrieve all flights


# Initialize a list to hold the extracted data
data_airports = []

# Loop over all airports
for airport in airports:
        # Retrieve flight details for each flight
    try:
        
        # Extract relevant fields
        data_airports.append({
            "airportID": airport.iata, # object
            "airportName": airport.name, #object
            "airportLat": airport.latitude, #object
            "airportLong": airport.longitude, #object
            "airportCountryName": airport.country #object,
        })
    except Exception as e:
        # Handle errors gracefully
        print(f"Error retrieving flight details for flight {airport}: {e}")

# Create a DataFrame
df_airports = pd.DataFrame(data_airports)

# Display the DataFrame
df_airports


,airportID,airportName,airportLat,airportLong,airportCountryName
0,LECO,A Coruna Airport,43.302059,-8.377250,Spain
1,EDKA,Aachen Merzbruck Airport,50.821899,6.184759,Germany
2,EKYT,Aalborg Airport,57.095112,9.855172,Denmark
3,EKAH,Aarhus Airport,56.303295,10.619129,Denmark
4,EKAC,Aarhus Sea Airport,56.151993,10.247725,Denmark
...,...,...,...,...,...
5811,ZUZY,Zunyi Xinzhou Airport,27.811167,107.245972,China
5812,LSZH,Zurich Airport,47.464722,8.549167,Switzerland
5813,HLZW,Zwara Airport,32.933334,12.083333,Libya
5814,EDRZ,Zweibrucken Airport,49.209518,7.400647,Germany


In [ ]:
import pandas as pd

# Retrieve all flights


# Initialize a list to hold the extracted data
data_airports = []

# Loop over all airports
for airport in airports:
        # Retrieve flight details for each flight
    try:
        airport_details = fr_api.get_airport_details(airport)  
        # Extract relevant fields
        data_airports.append({
            "airportID": airport_details['pluginData']['details']['code']['icao'], # object
            "airportName": airport_details['pluginData']['details']['name'], #object
            "airportLat": airport_details['pluginData']['details']['position']['latitude'], #object
            "airportLong": airport_details['pluginData']['details']['position']['longitude'], #object
            # "airportCountryCode": airport_details['pluginData']['details']['position']['country']['code'],
            "airportCountryName": airport_details['pluginData']['details']['position']['country']['name'] #object,
            # "airportContinentName": airport_details['pluginData']['details']['timezone']['name'],
            # "airportOrigineCountryID": flight_details['airport']['origin']['position']['country']['id'],
            # "airportOrigineContinent": flight_details['airport']['origin']['timezone']['name'],
            # "airportDestinationCode": flight_details['airport']['destination']['code']['icao'],
            # "airportDestinationLat": flight_details['airport']['destination']['position']['latitude'],
            # "airportDestinationLong": flight_details['airport']['destination']['position']['longitude'],
            # "airportDestinationCountryID": flight_details['airport']['destination']['position']['country']['id'],
            # "airportDestinationContinent": flight_details['airport']['destination']['timezone']['name']
        })
    except Exception as e:
        # Handle errors gracefully
        print(f"Error retrieving flight details for flight {flight}: {e}")

# Create a DataFrame
df_airports = pd.DataFrame(data_airports)

# Display the DataFrame
df_airports


### airlines

In [50]:
airlines = fr_api.get_airlines()
# Convert the list of dictionaries to a DataFrame
df_airlines = pd.DataFrame(airlines)
df_airlines

,Name,Code,ICAO
0,21 Air,2I,CSB
1,247 Aviation,,EMC
2,2Excel Aviation,,BRO
3,4 Airways,,DAK
4,40-Mile Air,Q5,MLA
...,...,...,...
2265,Zimex Aviation Austria,ZX,AZD
2266,Zipair,ZG,TZP
2267,Zooom,ZO,ZOM
2268,Zorex,,ORZ


In [53]:
zones = fr_api.get_zones()
zones

{'europe': {'tl_y': 72.57,
  'tl_x': -16.96,
  'br_y': 33.57,
  'br_x': 53.05,
  'subzones': {'poland': {'tl_y': 56.86,
    'tl_x': 11.06,
    'br_y': 48.22,
    'br_x': 28.26},
   'germany': {'tl_y': 57.92, 'tl_x': 1.81, 'br_y': 45.81, 'br_x': 16.83},
   'uk': {'tl_y': 62.61,
    'tl_x': -13.07,
    'br_y': 49.71,
    'br_x': 3.46,
    'subzones': {'london': {'tl_y': 53.06,
      'tl_x': -2.87,
      'br_y': 50.07,
      'br_x': 3.26},
     'ireland': {'tl_y': 56.22, 'tl_x': -11.71, 'br_y': 50.91, 'br_x': -4.4}}},
   'spain': {'tl_y': 44.36, 'tl_x': -11.06, 'br_y': 35.76, 'br_x': 4.04},
   'france': {'tl_y': 51.07, 'tl_x': -5.18, 'br_y': 42.17, 'br_x': 8.9},
   'ceur': {'tl_y': 51.39, 'tl_x': 11.25, 'br_y': 39.72, 'br_x': 32.55},
   'scandinavia': {'tl_y': 72.12, 'tl_x': -0.73, 'br_y': 53.82, 'br_x': 40.67},
   'italy': {'tl_y': 47.67, 'tl_x': 5.26, 'br_y': 36.27, 'br_x': 20.64}}},
 'northamerica': {'tl_y': 75,
  'tl_x': -180,
  'br_y': 3,
  'br_x': -52,
  'subzones': {'na_n': {'tl_y'

In [26]:
# Step 4: Ingest Zones
zones = fr_api.get_zones()  # Returns a nested dictionary of zones
data_zones = []
for continent, details in zones.items():
    if "subzones" in details:
        for subzone, sub_details in details["subzones"].items():
            data_zones.append({
                "continent": continent,
                "subzone": subzone,
                "tl_y": sub_details["tl_y"],
                "tl_x": sub_details["tl_x"],
                "br_y": sub_details["br_y"],
                "br_x": sub_details["br_x"]
            })
    else:
        data_zones.append({
            "continent": continent,
            "subzone": None,
            "tl_y": details["tl_y"],
            "tl_x": details["tl_x"],
            "br_y": details["br_y"],
            "br_x": details["br_x"]
        })

df_zones = pd.DataFrame(data_zones)


In [27]:
df_zones

,continent,subzone,tl_y,tl_x,br_y,br_x
0,europe,poland,56.86,11.06,48.22,28.26
1,europe,germany,57.92,1.81,45.81,16.83
2,europe,uk,62.61,-13.07,49.71,3.46
3,europe,spain,44.36,-11.06,35.76,4.04
4,europe,france,51.07,-5.18,42.17,8.90
5,europe,ceur,51.39,11.25,39.72,32.55
6,europe,scandinavia,72.12,-0.73,53.82,40.67
7,europe,italy,47.67,5.26,36.27,20.64
8,northamerica,na_n,72.82,-177.97,41.92,-52.48
9,northamerica,na_c,54.66,-134.68,22.16,-56.91
